In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, StratifiedShuffleSplit, KFold, cross_val_score
from sklearn.datasets import make_classification

from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import joblib

In [3]:
data=pd.read_csv('/Users/mucheng/CS6220HM/FinalProject/Final_Project/Task_3.ipynb', header=0, encoding='latin-1')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2


In [ ]:
data

In [ ]:
data['Delivery Status'].unique()

In [ ]:
tez = data.set_index('Category Id').to_dict()['Category Name']
print(tez)

## Fill the missing values

In [ ]:
bool_res = data.isnull()
tot=0
for c in data.columns:
    if True in bool_res[c].values:
        data[c].fillna('Missing', inplace=True)

In [ ]:
columns_list=['Category Id','Latitude','Longitude','Market','Shipping Mode','Delivery Status']
df=data[columns_list]

## Encode

In [ ]:
one_hot_df = pd.get_dummies(df, prefix={'Category Id':'C', 'Market':'Market', 'Shipping Mode':'Mode'}, columns=['Category Id', 'Market', 'Shipping Mode'], drop_first=False)

## Class Proportion

In [ ]:
print(f"PROPORTION OF TARGET IN THE ORIGINAL DATA\n{one_hot_df['Delivery Status'].value_counts() / len(one_hot_df)}\n\n")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
kwargs = dict(
    startangle = 90,
    colormap   = 'Pastel2',
    fontsize   = 13,
    explode    = (0.1,0.1,0.1,0.1),
    figsize    = (60,5),
    autopct    = '%1.1f%%',
    title      = 'Categorical Distributions'
)
plt.axis('off')
one_hot_df['Delivery Status'].value_counts().plot.pie(**kwargs)

In [ ]:
from sklearn import preprocessing
x=one_hot_df.loc[:, one_hot_df.columns != 'Delivery Status']
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)

In [ ]:
print(x.columns)

In [ ]:
y=one_hot_df.loc[:, one_hot_df.columns == 'Delivery Status']
skf=StratifiedKFold(n_splits=10, shuffle=True, random_state=11)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'criterion':['gini','entropy'],
    'max_depth':[0,4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]
}
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()

GS = GridSearchCV(dt,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',        # metric for scoring
                   cv=skf)

GS1 = GS.fit(x_scaled, y)

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv=skf)
GS2 = grid_search.fit(x_scaled, y.to_numpy().ravel())

In [ ]:
scores = [x for x in GS1.cv_results_['mean_test_score']]
scores = np.array(scores).reshape(len(parameters['criterion']), len(parameters['max_depth']))

for ind, i in enumerate(parameters['criterion']):
    plt.plot(parameters['max_depth'], scores[ind], label='C: ' + str(i))
plt.legend()
plt.xlabel('Max Depth')
plt.ylabel('Mean score')
plt.show()

In [ ]:
GS1.best_params_

In [ ]:
GS1.best_score_

In [ ]:
def train(cf, x, y):
    over = SMOTE(sampling_strategy='minority')
    under = RandomUnderSampler(sampling_strategy='majority')

    for train_index, test_index in skf.split(x, y):
        x_train_fold, x_test_fold = x[train_index], x[test_index]
        y_train_fold, y_test_fold = y.loc[train_index].values.ravel(), y.loc[test_index].values.ravel()
        print("fitting----")
        cf.fit(x_train_fold, y_train_fold)
        print("done")
        y_pred=cf.predict(x_test_fold)
        print(classification_report(y_test_fold, y_pred))
        plot_confusion_matrix(cf, x_test_fold, y_test_fold, display_labels=['Advance', 'Late', 'Cancel', 'On Time'])

dt = DecisionTreeClassifier()
gb = GradientBoostingClassifier()
rf = RandomForestClassifier()
bg = BaggingClassifier()
kb = KNeighborsClassifier()
train(bg, x_scaled, y)
# over = SMOTE(sampling_strategy='minority')
# under = RandomUnderSampler(sampling_strategy='majority')
# steps = [('over', over), ('under', under), ('model', rf)]
# pipeline = Pipeline(steps=steps)
# x1=x_scaled
# y1=y.to_numpy().ravel()
# cross_val_score(pipeline, x1, y1, cv=skf, scoring='accuracy')
# cross_val_score(bg, x1, y1, cv=skf)

In [ ]:
joblib.dump(dt, 'delivery_classifier.joblib')